<a href="https://colab.research.google.com/github/romerofernandes/class_fault/blob/main/class_fault.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparação e análise

In [16]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score

[0 0 0 0] - No Fault

[1 0 0 1] - LG fault (Between Phase A and Gnd)

[0 0 1 1] - LL fault (Between Phase A and Phase B)

[1 0 1 1] - LLG Fault (Between Phases A,B and ground)

[0 1 1 1] - LLL Fault(Between all three phases)

[1 1 1 1] - LLLG fault( Three phase symmetrical fault)

In [17]:
df_fault1 = pd.read_csv('/content/classData.csv')

In [18]:
df_fault2 = pd.read_csv('/content/detect_dataset.csv')
df_fault2.drop(columns = ['Unnamed: 7', 'Unnamed: 8'], inplace =True)
df_fault2.rename(columns={'Output (S)': 'fault'}, inplace=True)
df_fault2 = df_fault2[['Ia', 'Ib', 'Ic', 'Va', 'Vb', 'Vc', 'fault']]

In [19]:
df_fault2.head()

,Ia,Ib,Ic,Va,Vb,Vc,fault
0,-170.472196,9.219613,161.252583,0.054490,-0.659921,0.605431,0
1,-122.235754,6.168667,116.067087,0.102000,-0.628612,0.526202,0
2,-90.161474,3.813632,86.347841,0.141026,-0.605277,0.464251,0
3,-79.904916,2.398803,77.506112,0.156272,-0.602235,0.445963,0
4,-63.885255,0.590667,63.294587,0.180451,-0.591501,0.411050,0


In [22]:
df_fault1['cod_fault'] = df_fault1.G.astype(str) + df_fault1.C.astype(str) + df_fault1.B.astype(str) + df_fault1.A.astype(str)

In [23]:
df_fault1['fault'] = df_fault1.apply(lambda x: 0 if x.cod_fault == '0000' else 1, axis=1)

In [24]:
df_fault1.head()

,G,C,B,A,Ia,Ib,Ic,Va,Vb,Vc,cod_fault,fault
0,1,0,0,1,-151.291812,-9.677452,85.800162,0.400750,-0.132935,-0.267815,1001,1
1,1,0,0,1,-336.186183,-76.283262,18.328897,0.312732,-0.123633,-0.189099,1001,1
2,1,0,0,1,-502.891583,-174.648023,-80.924663,0.265728,-0.114301,-0.151428,1001,1
3,1,0,0,1,-593.941905,-217.703359,-124.891924,0.235511,-0.104940,-0.130570,1001,1
4,1,0,0,1,-643.663617,-224.159427,-132.282815,0.209537,-0.095554,-0.113983,1001,1


In [45]:
df_fault = pd.concat([df_fault1[['Ia', 'Ib', 'Ic', 'Va', 'Vb', 'Vc', 'fault']], df_fault2], ignore_index=True)

In [48]:
df_fault.shape[0]

18862

In [47]:
df_fault_test = df_fault.iloc[-1000:]
df_fault.drop(index=[*range(df_fault.shape[0]-1, df_fault.shape[0] - 1001, -1)], inplace=True)

In [63]:
df_fault_test.head()

,Ia,Ib,Ic,Va,Vb,Vc,fault
18862,48.161479,29.768698,-81.086667,-0.166331,0.603835,-0.437503,0
18863,47.703773,30.760823,-81.620782,-0.175520,0.606284,-0.430764,0
18864,47.232229,31.753535,-82.141648,-0.184669,0.608589,-0.423920,0
18865,46.746964,32.746589,-82.649134,-0.193775,0.610748,-0.416972,0
18866,46.248098,33.739739,-83.143114,-0.202837,0.612761,-0.409924,0


##Treinamento e teste

In [49]:
df_fault.head()

,Ia,Ib,Ic,Va,Vb,Vc,fault
0,-151.291812,-9.677452,85.800162,0.400750,-0.132935,-0.267815,1
1,-336.186183,-76.283262,18.328897,0.312732,-0.123633,-0.189099,1
2,-502.891583,-174.648023,-80.924663,0.265728,-0.114301,-0.151428,1
3,-593.941905,-217.703359,-124.891924,0.235511,-0.104940,-0.130570,1
4,-643.663617,-224.159427,-132.282815,0.209537,-0.095554,-0.113983,1


In [50]:
X = df_fault[['Ia', 'Ib', 'Ic', 'Va', 'Vb', 'Vc']]
y = df_fault['fault']

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [52]:
classif_fault = RandomForestClassifier()

In [53]:
classif_fault.fit(X_train, y_train)

RandomForestClassifier()

In [54]:
y_predict = classif_fault.predict( X_test)

In [55]:
confusion_matrix(y_test, y_predict)

array([[1569,    2],
       [   4, 2198]])

In [56]:
precision_score(y_test, y_predict, average=None)

array([0.99745709, 0.99909091])

In [60]:
y_predict2 = classif_fault.predict(df_fault_test[['Ia', 'Ib', 'Ic', 'Va', 'Vb', 'Vc']])

In [61]:
confusion_matrix(df_fault_test['fault'], y_predict2)

array([[1000]])

In [ ]:
precision_score(df_fault_test['Output (S)'], y_predict2, average=None)

array([1.        , 0.74755169])